In [1]:
from syft.core.tensor.passthrough import PassthroughTensor
from typing import Union,Optional,Any,Tuple
import numpy as np
from syft.core.tensor.config import DEFAULT_INT_NUMPY_TYPE
from syft.core.tensor.config import DEFAULT_FLOAT_NUMPY_TYPE


/home/azureuser/.local/share/virtualenvs/python39-2vM0bQN-/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
class FixedPrecisionTensor(PassthroughTensor):
    def __init__(
        self,
        value: Union[int, float, np.ndarray] = None,
        base: int = 2,
        precision: int = 16,
    ) -> None:
        self._base = base
        self._precision = precision
        self._scale = base**precision
        if value is not None:
            super().__init__(self.encode(value))
        else:
            super().__init__(None)

    def encode(self, value: Union[int, float, np.ndarray]) -> np.ndarray:
        encoded_value = np.array(self._scale * value, DEFAULT_INT_NUMPY_TYPE)
        return encoded_value

    @property
    def dtype(self) -> np.dtype:
        return getattr(self.child, "dtype", None)

    @property
    def shape(self) -> Optional[Tuple[int, ...]]:
        return getattr(self.child, "shape", None)

    def decode(self) -> Any:
        value = self.child
        scale = self._scale

        correction = (value < 0).astype(DEFAULT_INT_NUMPY_TYPE)

        dividend = np.trunc(value / scale - correction)
        remainder = value % scale
        remainder += (
            (remainder == 0).astype(DEFAULT_INT_NUMPY_TYPE) * scale * correction
        )
        value = (
            dividend.astype(DEFAULT_FLOAT_NUMPY_TYPE)
            + remainder.astype(DEFAULT_FLOAT_NUMPY_TYPE) / scale
        )
        return value

    def sanity_check(
        self, other: Union["FixedPrecisionTensor", int, float, np.ndarray]
    ) -> "FixedPrecisionTensor":
        if isinstance(other, FixedPrecisionTensor):
            if self._base != other._base or self._precision != other._precision:
                raise ValueError(
                    f"Base:{self.base,other.base} and Precision: "
                    + f"{self.precision, other.precision} should be same for "
                    + "computation on FixedPrecisionTensor"
                )
        elif isinstance(other, (int,float,np.ndarray)):
            other = FixedPrecisionTensor(
                value=other, base=self._base, precision=self._precision
            )
        else:
            raise ValueError(f"Invalid type for FixedPrecisionTensor: {type(other)}")

        return other

    def __add__(self, other: Any):
        res = FixedPrecisionTensor(base=self._base, precision=self._precision)
        other = self.sanity_check(other)
        res.child = self.child + other.child
        return res

    def __sub__(self, other: Any):
        res = FixedPrecisionTensor(base=self._base, precision=self._precision)
        other = self.sanity_check(other)
        res.child = self.child - other.child
        return res

    def __mul__(self, other: Any): 
        res = FixedPrecisionTensor(base=self._base, precision=self._precision)
        other = self.sanity_check(other)
        res.child = self.child * other.child
        res = res / self._scale
        return res

    def __lt__(self, other: Any):
        other = self.sanity_check(other)
        value = (self.child < other.child) * 1

        res = FixedPrecisionTensor(
            value=value, base=self._base, precision=self._precision
        )
        return res

    def __gt__(self, other: Any):
        other = self.sanity_check(other)
        value = (self.child > other.child) * 1
        res = FixedPrecisionTensor(
            value=value, base=self._base, precision=self._precision
        )
        return res

    def __truediv__(
        self, other: Union[int, np.integer, "FixedPrecisionTensor"]
    ) -> "FixedPrecisionTensor":
        if isinstance(other, FixedPrecisionTensor):
            raise ValueError("We do not support Private Division yet.")

        res = FixedPrecisionTensor(base=self._base, precision=self._precision)
        if isinstance(self.child, np.ndarray) or np.isscalar(self.child):
            res.child = np.trunc(self.child / other).astype(DEFAULT_INT_NUMPY_TYPE)
        else:
            res.child = self.child / other
        return res

    def sum(
        self, axis: Optional[Union[int, Tuple[int, ...]]] = None
    ) :
        res = FixedPrecisionTensor(base=self._base, precision=self._precision)
        if isinstance(self.child, np.ndarray):
            res.child = np.array(self.child.sum(axis=axis))
        else:
            res.child = self.child.sum(axis=axis)
        return res

        

In [37]:
def sign(data):
    return (data>0) + (data<0)*-1

def modulus(data):
    return sign(data) * data

def exp(value, iterations = 8):
    result = (value / 2**iterations) + 1
    for _ in range(iterations):
        result = result * result
    return result

def reciprocal(data, method = "NR", nr_iters = 10):
    method = method.lower()
    if method == "nr":
        new_data = modulus(data)
        result = exp(new_data * -1 + 0.5) * 3 + 0.003
        for i in range(nr_iters):
            result = result * 2 - result * result * new_data
        return result * sign(data)
    elif method == "log":
        new_data = modulus(data)
        return exp(-1 * log(new_data)) * sign(data)
    else:
        raise ValueError(f"Invalid method {method} given for reciprocal function")
    

def softmax(data):
    # max_val = data.child.max()
    # res = FixedPrecisionTensor(base = data._base,precision=data._precision)
    # res.child = data.child #-max_val
    numerator = exp(data)
    denominator = numerator.sum()
    inv = reciprocal(denominator)
    return numerator * inv




In [38]:
val = np.array([-0.125,.542,.5614,-.7518])


In [39]:
a = FixedPrecisionTensor(val)

In [40]:
a

FixedPrecisionTensor(child=[ -8192  35520  36791 -49269])

In [41]:
a.decode()

array([-0.125     ,  0.54199219,  0.56138611, -0.75178528])

In [42]:
softmax(a).decode()

array([0.18345642, 0.35588074, 0.36300659, 0.09793091])

In [43]:
np.exp(val)/np.exp(val).sum()

array([0.18284098, 0.35624434, 0.36322295, 0.09769172])

In [ ]:
new_data = modulus(a)
# result = exp(new_data * -1 + 0.5) * 3 + 0.003
result = 3 * exp((2 * new_data-1)*-1) + 0.003

In [ ]:
2**2154